In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import chaospy
import numpoly
import yaml
import numpy as np
import pandas as pd
import sys, os
from numpy.random import random

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use(["bmh", "../matplotlibrc"])

from sklearn.model_selection import train_test_split

In [ ]:
sys.path.insert(0, os.getcwd() + "/../scripts")
import _helpers as h
import _plotters as p
from surrogate import build_surrogate
from neural_network import build_neural_network
from sobol import calculate_sobol

## PCE Surrogate Modelling

In [ ]:
with open("../config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

In [ ]:
with open("../config.pypsaeur.yaml", 'r') as stream:
    TECH_COLORS = yaml.safe_load(stream)["plotting"]["tech_colors"]

In [ ]:
datafile = "../results/capacities-50halton.csv"
order = 3

In [ ]:
dataset = h.load_dataset(datafile)
distribution = h.NamedJ(config["uncertainties"])

In [ ]:
train_set, test_set = train_test_split(dataset, **config["train_test_split"])

In [ ]:
surrogate = build_surrogate(order, distribution, train_set)

In [ ]:
train_samples = h.multiindex2df(train_set.index)
train_predictions = h.build_pce_prediction(surrogate, train_samples)

test_samples = h.multiindex2df(test_set.index)
test_predictions = h.build_pce_prediction(surrogate, test_samples)

### Evaluation

In [ ]:
p.plot_histograms(dataset, [train_predictions, test_predictions])

In [ ]:
dataset.mean()

In [ ]:
h.calculate_errors(train_predictions, train_set).mean()

In [ ]:
h.calculate_errors(test_predictions, test_set).mean()

### Sensitivity Analysis

In [ ]:
sobol_t = calculate_sobol(surrogate, distribution)
sobol_t

In [ ]:
sobol_m = calculate_sobol(surrogate, distribution, sobol='m')
sobol_m

In [ ]:
def calculate_sobol_m2(surrogate, distribution, decimals=3):
    sobol = chaospy.Sens_m2(surrogate, distribution.J).round(decimals)
    return pd.DataFrame(sobol, index=distribution.names, columns=distribution.names)

In [ ]:
sobol_m2_tsc = calculate_sobol_m2(surrogate["tsc"], distribution)

In [ ]:
sobol_m2_transmission = calculate_sobol_m2(surrogate["transmission"], distribution)

In [ ]:
sobol_m2_solar = calculate_sobol_m2(surrogate["solar"], distribution)

In [ ]:
def plot_sobol_m2(sobol, title="", fn=None):
    fig, ax = plt.subplots(figsize=(3,3))
    mask=np.triu(np.ones(sobol.shape)).astype(np.bool)
    sns.heatmap(sobol, mask=mask, square=True, cmap="Blues",
            vmax=.2,
            vmin=0,
            annot=True,
            fmt=".2f",
            cbar=False,)
    plt.title(title)
    if fn is not None:
        plt.savefig(fn, bbox_inches='tight')

In [ ]:
plot_sobol_m2(sobol_m2_tsc, "TSC", "sobol-tsc-m2.pdf")

In [ ]:
plot_sobol_m2(sobol_m2_transmission, "transmission", "sobol-transmission-m2.pdf")

In [ ]:
plot_sobol_m2(sobol_m2_solar, "solar", "sobol-solar-m2.pdf")

In [ ]:
p.plot_sobol(sobol_t, fn='sobol-t.pdf')

In [ ]:
p.plot_sobol(sobol_m, fn='sobol-m.pdf')

In [ ]:
p.plot_sobol(sobol_t - sobol_m, fn='sobol-diff.pdf')

In [ ]:
def plot_sobol_bar(sobol, relative=True, fn=None):

    fig, ax = plt.subplots(figsize=(5,3))

    colors = [TECH_COLORS[s] for s in sobol_t.index]
    
    if relative:
        sobol = sobol / sobol.sum()

    sobol.T.plot.bar(ax=ax, stacked=True, color=colors)
    plt.legend(bbox_to_anchor=(1,1.01), ncol=1)
    plt.ylim([0,1])

    if fn is not None:
        plt.savefig(fn, bbox_inches='tight')

In [ ]:
plot_sobol_bar(sobol_m, relative=False, fn="sobol-m-bar.pdf")

In [ ]:
plot_sobol_bar(sobol_t, relative=True, fn="sobol-t-bar.pdf")

In [ ]:
plot_sobol_bar(sobol_t - sobol_m, relative=False, fn="sobol-diff-bar.pdf")

## 2D-Plots

In [ ]:
def plot_2D(surrogate, variable, xname, yname, xsamples=(0.5,1.5,20), ysamples=(0.5,1.5,20), dataset=None, contour_handles=None, vmin=130, vmax=260, levels=25, fn=None):
    pass

In [ ]:
xs = np.linspace(*xsamples)
ys = np.linspace(*ysamples)

# TODO: pass fixed values in function
zpoly = surrogate[variable](q2=1, q3=1, q4=1)

# TODO: accessing variables
z = np.array([zpoly(q0=x, q1=ys) for x in xs])

if contour_handles is None:
    
    def idx():
        return int(np.round(random())) % 2

    rng = [vmin-5,vmax+5]
    dim1 = [rng[idx()] for i in range(ys.shape[0])]
    zdummy = np.array(xs.shape[0]*[dim1])
    
    contour_handles = plt.contourf(xs, ys, zdummy, levels=25, vmin=vmin, vmax=vmax)
    
    plt.close()
    
fig, ax = plt.subplots(figsize=(6,5))
    
plt.contourf(xs, ys, z, levels=contour_handles.levels)

cbar = plt.colorbar(contour_handles, label=variable)

plt.xlabel(f"{xname}-cost")
plt.ylabel(f"{yname}-cost")

if dataset is not None:
    df = dataset.reset_index().astype(float)
    plt.scatter(df[f"{xname}-cost"], df[f"{yname}-cost"], marker='.', s=5, alpha=0.2, color='grey')
    
if fn is not None:
    plt.savefig(fn, bbox_inches='tight')
    
plt.close()

In [ ]:
x = y = np.linspace(0.5,1.5,20)
z = surrogate["tsc"](q2=1, q3=1, q4=1)

Z = []
for i in x:
    Z.append(z(q0=i, q1=y))
Z = np.array(Z)

fig, ax = plt.subplots(figsize=(6,5))
plt.contourf(x, y, Z, levels=cs.levels)
cbar = fig.colorbar(cs, label="TSC", ax=ax)
plt.xlabel("offwind-cost")
plt.ylabel("onwind-cost")

df = dataset.reset_index().astype(float)
plt.scatter(df["onwind-cost"], df["offwind-cost"], marker='.', s=5, alpha=0.2, color='grey')

plt.savefig("2D-onwind-offwind-H2-100.pdf", bbox_inches='tight')

In [ ]:
x = y = np.linspace(0.5,1.5,20)
z = surrogate["tsc"](q2=1, q3=1.5, q4=1)

Z = []
for i in x:
    Z.append(z(q0=i, q1=y))
Z = np.array(Z)

fig, ax = plt.subplots(figsize=(6,5))
plt.contourf(x, y, Z, levels=cs.levels)
cbar = plt.colorbar(cs, label="TSC")
plt.xlabel("offwind-cost")
plt.ylabel("onwind-cost")


df = dataset.reset_index().astype(float)
plt.scatter(df["onwind-cost"], df["offwind-cost"], marker='.', s=5, alpha=0.2, color='grey')

plt.savefig("2D-onwind-offwind-H2-150.pdf", bbox_inches='tight')

In [ ]:
x = y = np.linspace(0.5,1.5,20)
z = surrogate["tsc"](q2=1, q3=0.5, q4=1)

Z = []
for i in x:
    Z.append(z(q0=i, q1=y))
Z = np.array(Z)

fig, ax = plt.subplots(figsize=(6,5))
plt.contourf(x, y, Z, levels=cs.levels)
cbar = plt.colorbar(cs, label="TSC")
plt.xlabel("offwind-cost")
plt.ylabel("onwind-cost")


df = dataset.reset_index().astype(float)
plt.scatter(df["onwind-cost"], df["offwind-cost"], marker='.', s=5, alpha=0.2, color='grey')

plt.savefig("2D-onwind-offwind-H2-050.pdf", bbox_inches='tight')

In [ ]:
x = y = np.linspace(0.5,1.5,20)
z = surrogate["tsc"](q0=1, q2=1, q4=1)

Z = []
for i in x:
    Z.append(z(q1=i, q3=y))
Z = np.array(Z)

fig, ax = plt.subplots(figsize=(6,5))
plt.contourf(x, y, Z, levels=cs.levels)
cbar = plt.colorbar(cs, label="TSC")
plt.xlabel("H2-cost")
plt.ylabel("offwind-cost")


df = dataset.reset_index().astype(float)
plt.scatter(df["onwind-cost"], df["offwind-cost"], marker='.', s=5, alpha=0.2, color='grey')

plt.savefig("2D-H2-offwind.pdf", bbox_inches='tight')

In [ ]:
x = y = np.linspace(0.5,1.5,20)
z = surrogate["tsc"](q0=1, q2=1, q3=1)

Z = []
for i in x:
    Z.append(z(q1=i, q4=y))
Z = np.array(Z)

fig, ax = plt.subplots(figsize=(6,5))
plt.contourf(x, y, Z, levels=cs.levels, vmin=135, vmax=250)
cbar = plt.colorbar(cs, label="TSC")
plt.xlabel("battery-cost")
plt.ylabel("offwind-cost")

df = dataset.reset_index().astype(float)
plt.scatter(df["onwind-cost"], df["offwind-cost"], marker='.', s=5, alpha=0.2, color='grey')

plt.savefig("2D-battery-offwind.pdf", bbox_inches='tight')

## 1D Plots

In [ ]:
def plot_1D(surrogate, variable, parameter, coords, distribution, sample=10000, dataset=None, color_by_var=True, fn=None):
    
    poly = surrogate[variable]
    symbol = f"q{distribution.mapping[parameter]}"
    if color_by_var:
        color = TECH_COLORS[var]
    else:
        color = TECH_COLORS[parameter]
    percentiles = [5,25,50,75,95]
    
    P = []
    for coord in coords:
        symvalues = {symbol: coord}
        P.append(chaospy.Perc(poly(**symvalues), percentiles, distribution.J, sample=sample))
    P = np.array(P)
    
    fig, ax = plt.subplots(figsize=(3,3))
    plt.plot(coords, P[:,2], linewidth=1, label="Q50", color=color);
    plt.fill_between(coords, P[:,1], P[:,3], alpha=0.2, label="Q25/Q75", color=color)
    plt.fill_between(coords, P[:,0], P[:,4], alpha=0.2, label="Q5/Q95", color='grey')
    plt.ylabel(variable)
    plt.xlabel(f"{parameter}-cost")
    plt.legend(frameon=False)
    
    if var == "tsc":
        plt.ylim([130,270])
    elif var in ["H2", "battery"]:
        plt.ylim([0,250])
    elif var == "transmission":
        plt.ylim([0,900])
    else:
        plt.ylim([0,1400])
    
    if dataset is not None:
        df = dataset.reset_index().astype(float)
        plt.scatter(df[f"{parameter}-cost"], df[variable], marker='.', alpha=0.1, color='grey')

    if fn is not None:
        plt.savefig(fn, bbox_inches='tight')
        
    plt.close()

In [ ]:
# TODO use multiprocessing
for var in set(dataset.columns):
    for param in distribution.names:
        print(param, var)
        fn = f"graphics/1D/1D-{var}-{param}.pdf"
        if os.path.isfile(fn):
            continue
        plot_1D(surrogate, var, param, np.linspace(0.5,1.5,25), distribution, sample=20000, dataset=dataset, fn=fn)

## Pure Machine Learning with `sklearn`

In [ ]:
with open("../config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

In [ ]:
neural_network = build_neural_network(train_set, config["neural_network"])

In [ ]:
train_predictions = h.build_ann_prediction(neural_network, train_samples, train_set)
test_predictions = h.build_ann_prediction(neural_network, test_samples, test_set)

### Evaluation

In [ ]:
p.plot_histograms(dataset, [train_predictions, test_predictions])

In [ ]:
h.calculate_errors(train_predictions, train_set).mean()

In [ ]:
h.calculate_errors(test_predictions, test_set).mean()

## Multi-fidelity approach

- many more samples in very low resolution model

## Another Easy Benchmark to Beat:

- Surrogate is obtained from MC sampling for 37 nodes and 6-hourly resolution

In [ ]:
# TODO